In [ ]:
from os import chdir
from sys import path

from kedro.config import OmegaConfigLoader
from kedro.io import DataCatalog

chdir("..")
path.append("src/")
conf_loader = OmegaConfigLoader(conf_source=".")
catalog = DataCatalog.from_config(conf_loader["catalog"])

In [ ]:
import os

import yaml

# load RAG assets including vector db into a temp dir
temp_path = catalog.load('rag_deployment_assets')
# load custom.py into this notebook
exec((temp_path / "custom.py").read_text())
# load RAG parameters
params = {param['fieldName']: param.get('defaultValue', None) for param in yaml.safe_load(catalog.load('prep_user_rag_custom_model.model_metadata'))['runtimeParameterDefinitions']}
# add your OPENAI API KEY
params['openai_api_key'] = os.environ['OPENAI_API_KEY']
# load the RAG chain
chain = get_chain(str(temp_path.resolve()), **params)

In [ ]:
# insert your question of interest
chain_output = chain.invoke(
    {
        "question": 'hello!',
        "chat_history": [],
    }
)

In [ ]:
import pprint

pprint.pprint(chain_output)

In [ ]:
from shutil import rmtree

# clean up the temp dir
rmtree(temp_path)